In [45]:
#|export
"""
Run simply with
$ pytest
"""
import pytest

import os
import subprocess
import numpy as np
import torch
import struct

from export import export_model, export_modelq8
from train import load

#this is predicted wrong on fastai resnet18
#file_path = "test/data/imagenette2/val_transformed/0/113"
file_path = "data/imagenette2/val_transformed/0/2"

In [46]:
#|export
def calculate_reference_values(model_name, file_path):
    # calculate reference values using Python model
    model = load(model_name)
    with open(file_path, "rb") as f:
        sizeof_float, nch, h, w = 4, 3, 224, 224
        image = torch.tensor(struct.unpack("f"*(nch*h*w), f.read(sizeof_float*nch*h*w))).view(1, nch,h,w)
        ref = model(image).detach()
        ref = torch.nn.functional.softmax(ref, dim=1).view(-1).numpy() # python model output
    return ref

In [47]:
#|export
def execute(command):
    d = "test_outputs"
    os.makedirs(d, exist_ok=True)
    err = os.path.join(d, "err.txt")
    out = os.path.join(d, "stdout.txt")
    with open(err, mode="w") as fe:
        with open(out, mode="w") as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res = np.loadtxt(out)

    return res

@pytest.mark.parametrize("model_name", ["resnet18", "resnet34", "resnet50"])
def test_runfiles(model_name, quantized=False, file_path=file_path):
    """ test run.c and runq.c works with an acceptable tolerance """
    # run vanilla model in test mode
    ref = calculate_reference_values(model_name, file_path="data/imagenette2/val_transformed/0/2")
    export_model(model_name)
    compile = [f'make compile_{model_name}']
    subprocess.run(compile, shell=True)
    command = ["./run", "model.bin", file_path]
    res = execute(command)
    assert np.allclose(res, ref, atol=1e-5, rtol=0), "run.c: Probabilities are not close."

    if quantized:
        # run quantized model test with group size = 1 and 2 in test mode
        export_modelq8(file_path="modelq8_1.bin", gs=1)
        resq1 = execute(["./runq", "modelq8_1.bin", file_path])
        assert np.allclose(resq1, ref, atol=1e-5, rtol=0), "runq.c (group size = 1): Probabilities are not close."

        export_modelq8(file_path="modelq8_2.bin", gs=2)
        resq2 = execute(["./runq", "modelq8_2.bin", file_path])
        assert np.allclose(resq2, ref, atol=1e-2, rtol=0), "runq.c (group size = 2):Probabilities are not close."

    print("Done")

In [49]:
test_runfiles("resnet34", False)

wrote model.bin
clang -Os -Wall -DRESNET34 run.c  func.c -lm -o run
clang -Os -Wall runq.c func.c -lm -o runq


run.c:235:13: warning: unused function 'resnet18' [-Wunused-function]
  235 | static void resnet18(ConvConfig *cc, LinearConfig *lc, float *p, Model *m,
      |             ^~~~~~~~
run.c:279:13: warning: unused function 'resnet50' [-Wunused-function]
  279 | static void resnet50(ConvConfig *cc, LinearConfig *lc, float *p, Model *m,
      |             ^~~~~~~~
2 warnings generated.


Done


In [104]:
def hook_fn(module, input, output):
    print(output)
    print(output.shape)
    return output


model = load("resnet50")
fs = model[0][3]
first_resnet_layer = model[0][4]
print(len(first_resnet_layer))
second_resnet_layer = model[0][5]
last_resnet_layer = model[0][7]
pool_layer = model[1][0]
bn_1_layer = model[1][2]
ln1_layer = model[1][4]
bn2_layer = model[1][6]
ln2_layer = model[1][8]

file_path = "data/imagenette2/val_transformed/0/2"
with open(file_path, "rb") as f:
    sizeof_float, nch, h, w = 4, 3, 224, 224
    image = torch.tensor(struct.unpack("f"*(nch*h*w), f.read(sizeof_float*nch*h*w))).view(1, nch,h,w)
    hook = first_resnet_layer.register_forward_hook(hook_fn)
    out = model(image).detach()
    output = torch.nn.functional.softmax(out, dim=1).view(-1).numpy()
    print(output)
    hook.remove()

3
tensor([[[[1.1400, 0.8885, 1.4719,  ..., 0.5663, 0.9684, 2.1019],
          [2.1637, 1.6843, 0.7259,  ..., 1.3679, 1.1357, 2.0868],
          [0.2753, 0.8641, 0.1392,  ..., 1.0107, 1.1980, 1.8648],
          ...,
          [1.3349, 1.1786, 0.8576,  ..., 1.0482, 0.2410, 0.4808],
          [1.1318, 0.7934, 1.5452,  ..., 0.7654, 0.7076, 0.0306],
          [2.7300, 2.4416, 1.9129,  ..., 1.9120, 2.1227, 2.1891]],

         [[1.3740, 1.1486, 0.9849,  ..., 1.0718, 1.1977, 0.8783],
          [0.5917, 0.4507, 1.5132,  ..., 1.3332, 0.9195, 0.3703],
          [0.9547, 0.9663, 1.6444,  ..., 0.9273, 1.0539, 0.8050],
          ...,
          [0.5531, 0.7749, 1.0821,  ..., 1.4570, 1.2188, 0.6488],
          [2.1351, 1.9420, 0.9674,  ..., 2.4470, 0.9771, 1.7373],
          [0.8505, 1.3167, 1.5605,  ..., 1.3034, 1.0526, 0.8555]],

         [[0.0000, 0.0000, 0.0000,  ..., 2.5231, 0.0000, 0.0000],
          [0.6186, 0.8010, 0.0000,  ..., 0.0000, 1.8106, 1.4587],
          [0.8934, 0.0000, 0.0000,  ...,

In [103]:
export_model("resnet50")
model = load("resnet50")
conv_layers = [layer for layer in model.modules() if isinstance(layer, torch.nn.Conv2d)]
print(len(conv_layers))

wrote model.bin
53
